In [ ]:
import pandas as pd

# ① 데이터 불러오기 ― 예시는 Excel, CSV일 때는 read_csv 사용
df = pd.read_excel('풍력발전.xls')      # 또는 pd.read_csv('발전설비목록.csv')

# ② A열에서 '풍력'이라는 글자가 들어간 행만 남기기
mask = df['발전구분'].str.contains('풍력',  # 찾을 키워드
                            case=False,  # 대소문자 무시(한글에는 거의 영향 없음)
                            na=False)    # NaN(결측값)은 False 처리
filtered = df[mask].copy()

# ③ 필요하다면 인덱스를 다시 매기고, 파일로 저장
filtered = filtered.reset_index(drop=True)
filtered.to_excel('풍력_필터링.xlsx', index=False)   # 또는 to_csv

In [57]:
filtered

,발전구분,호기,일자,1시 발전량(MWh),2시 발전량(MWh),3시 발전량(MWh),4시 발전량(MWh),5시 발전량(MWh),6시 발전량(MWh),7시 발전량(MWh),...,21시 발전량(MWh),22시 발전량(MWh),23시 발전량(MWh),24시 발전량(MWh),총량(MW),평균(MW),최대(시간별),최소(시간별),최대,최소
0,삼천포풍력,1,2025-03-31 00:00:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0,0.00,0.000,0.000,0.00
1,영흥풍력,1,2025-03-31 00:00:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,388.854,381.894,433.857,449.860,2680.928,112,449.86,33.999,449.860,0.00
2,영흥풍력,2,2025-03-31 00:00:00,24.960,0.000,0.000,0.000,0.000,0.000,11.520,...,2666.880,2014.080,2421.120,1096.320,24153.600,1006,4694.40,1.920,4694.400,0.00
3,군위 화산풍력,1,2025-03-31 00:00:00,3760.421,4927.116,4522.147,4049.684,3895.411,4329.305,2738.358,...,202.484,96.421,9.642,0.000,39195.157,1633,4927.11,9.642,4927.116,0.00
4,어음풍력,1,2025-03-31 00:00:00,3741.752,3533.152,4262.216,4196.472,3179.288,2373.224,2720.928,...,1089.424,2054.304,2256.184,2331.672,54816.664,2284,4262.21,516.880,4262.216,516.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,삼천포풍력,1,2025-03-07 00:00:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0,0.00,0.000,0.000,0.00
121,영흥풍력,1,2025-03-07 00:00:00,125.769,122.683,84.502,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,6889.324,287,3526.53,84.502,3526.530,0.00
122,영흥풍력,2,2025-03-07 00:00:00,3327.360,2313.600,1234.560,167.040,113.280,57.600,0.000,...,218.880,3.840,0.000,0.000,14983.680,624,3327.36,3.840,3327.360,0.00
123,군위 화산풍력,1,2025-03-07 00:00:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5264.589,5524.926,4686.063,2921.558,23931.705,997,5524.92,231.411,5524.926,0.00


In [58]:
unique_count = filtered['발전구분'].unique()
unique_count


array(['삼천포풍력', '영흥풍력', '군위 화산풍력', '어음풍력'], dtype=object)

In [59]:
# 'ㅇㅇ풍력발전' 데이터만 필터링
#일단 '군위화산풍력'
samcheonpo_df = df[df['발전구분'] == '군위 화산풍력'].copy()

# '일자'를 datetime 형식으로 변환
samcheonpo_df['일자'] = pd.to_datetime(samcheonpo_df['일자'])

# 시간별 발전량 열 추출 및 melt로 시계열 변환
hourly_columns = [col for col in samcheonpo_df.columns if '시 발전량' in col]

# '일자'와 시간별 발전량 열을 melt해서 시계열 데이터로 변환
samcheonpo_melted = samcheonpo_df.melt(
    id_vars=['일자', '호기'],
    value_vars=hourly_columns,
    var_name='시간',
    value_name='발전량(MWh)'
)

# '시간'에서 숫자만 추출하여 시각(datetime) 조합
samcheonpo_melted['시'] = samcheonpo_melted['시간'].str.extract(r'(\d+)').astype(int)
samcheonpo_melted['시각'] = samcheonpo_melted['일자'] + pd.to_timedelta(samcheonpo_melted['시'] - 1, unit='h')

# 필요한 열만 정리
time_series_df = samcheonpo_melted[['시각', '발전량(MWh)', '호기']].sort_values(by='시각')

In [60]:
time_series_df

,시각,발전량(MWh),호기
24,2025-03-07 00:00:00,0.000,1
49,2025-03-07 01:00:00,0.000,1
74,2025-03-07 02:00:00,0.000,1
99,2025-03-07 03:00:00,0.000,1
124,2025-03-07 04:00:00,0.000,1
...,...,...,...
475,2025-03-31 19:00:00,77.137,1
500,2025-03-31 20:00:00,202.484,1
525,2025-03-31 21:00:00,96.421,1
550,2025-03-31 22:00:00,9.642,1


In [ ]:
#기상데이터 추가(군위의 경우 영청(281) 데이터 사용)
#https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36
weather_df = pd.read_excel("기상데이터.xlsx")

# 데이터 미리보기
weather_df.head()

,지점,지점명,일시,풍속(m/s),풍속 QC플래그,풍향(16방위),풍향 QC플래그
0,281,영천,2025-03-07 00:00:00,0.0,NaN,0,NaN
1,281,영천,2025-03-07 01:00:00,0.6,NaN,270,NaN
2,281,영천,2025-03-07 02:00:00,0.6,NaN,270,NaN
3,281,영천,2025-03-07 03:00:00,0.4,NaN,0,NaN
4,281,영천,2025-03-07 04:00:00,0.2,NaN,0,NaN


In [62]:
# '일시' 컬럼을 datetime으로 변환
weather_df['일시'] = pd.to_datetime(weather_df['일시'])

# 컬럼명 통일을 위해 '일시' -> '시각'으로 변경
weather_df.rename(columns={'일시': '시각'}, inplace=True)

# 발전 데이터와 병합 (시각 기준)
merged_df = pd.merge(time_series_df, weather_df, on='시각', how='left')
merged_df


,시각,발전량(MWh),호기,지점,지점명,풍속(m/s),풍속 QC플래그,풍향(16방위),풍향 QC플래그
0,2025-03-07 00:00:00,0.000,1,281,영천,0.0,NaN,0,NaN
1,2025-03-07 01:00:00,0.000,1,281,영천,0.6,NaN,270,NaN
2,2025-03-07 02:00:00,0.000,1,281,영천,0.6,NaN,270,NaN
3,2025-03-07 03:00:00,0.000,1,281,영천,0.4,NaN,0,NaN
4,2025-03-07 04:00:00,0.000,1,281,영천,0.2,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...
595,2025-03-31 19:00:00,77.137,1,281,영천,1.5,NaN,90,NaN
596,2025-03-31 20:00:00,202.484,1,281,영천,2.0,NaN,70,NaN
597,2025-03-31 21:00:00,96.421,1,281,영천,0.8,NaN,110,NaN
598,2025-03-31 22:00:00,9.642,1,281,영천,0.8,NaN,50,NaN


In [63]:
# 원하는 순서 및 필요한 칼럼 재정렬
reordered_cols = ['시각', '지점명', '풍속(m/s)', '풍향(16방위)', '발전량(MWh)']
merged_df = merged_df[reordered_cols]

In [64]:
#지점 명을 풍력 발전기명으로 변경(되지만, 오류뜸)
merged_df['지점명'] = '군위 화산풍력'
merged_df

/var/folders/qj/004xgdtd15520sxppmd4bvvh0000gn/T/ipykernel_41658/2151021548.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['지점명'] = '군위 화산풍력'


,시각,지점명,풍속(m/s),풍향(16방위),발전량(MWh)
0,2025-03-07 00:00:00,군위 화산풍력,0.0,0,0.000
1,2025-03-07 01:00:00,군위 화산풍력,0.6,270,0.000
2,2025-03-07 02:00:00,군위 화산풍력,0.6,270,0.000
3,2025-03-07 03:00:00,군위 화산풍력,0.4,0,0.000
4,2025-03-07 04:00:00,군위 화산풍력,0.2,0,0.000
...,...,...,...,...,...
595,2025-03-31 19:00:00,군위 화산풍력,1.5,90,77.137
596,2025-03-31 20:00:00,군위 화산풍력,2.0,70,202.484
597,2025-03-31 21:00:00,군위 화산풍력,0.8,110,96.421
598,2025-03-31 22:00:00,군위 화산풍력,0.8,50,9.642


In [65]:
merged_df.to_excel('풍력_기상.xlsx', index=False)